In [1]:
import nltk; #nltk.download('stopwords')

In [2]:
import re

In [3]:
import numpy as np
import pandas as pd
from pprint import pprint

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

C:\Users\lara\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# spacy for lemmatization
import spacy

C:\Users\lara\Anaconda3\lib\site-packages\thinc\neural\train.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from .optimizers import Adam, SGD, linear_decay
C:\Users\lara\Anaconda3\lib\site-packages\thinc\check.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import defaultdict, Sequence, Sized, Iterable, Callable


In [7]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [8]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [9]:
data = pd.read_csv('train.tsv', sep='\t')

In [10]:
data.set_index('train_id', inplace=True)

In [11]:
data2 = data.item_description.values.tolist()

In [12]:
print(data2[1:10])

['This keyboard is in great condition and works like it came out of the box. All of the ports are tested and work perfectly. The lights are customizable via the Razer Synapse app on your PC.', 'Adorable top with a hint of lace and a key hole in the back! The pale pink is a 1X, and I also have a 3X available in white!', 'New with tags. Leather horses. Retail for [rm] each. Stand about a foot high. They are being sold as a pair. Any questions please ask. Free shipping. Just got out of storage', 'Complete with certificate of authenticity', 'Banana republic bottoms, Candies skirt with matching blazer,Amy Byers suit, Loft bottoms and cami top.', 'Size small but straps slightly shortened to fit xs, besides that, perfect condition', 'You get three pairs of Sophie cheer shorts size small and medium girls and two sports bra/boy shorts spandex matching sets in small and medium girls. All items total retail for [rm] in store and you can take him today for less than the price of one item at the st

In [13]:
data2 = [re.sub('\[rm\]', '', str(sent)) for sent in data2]
print(data2[1:10])

['This keyboard is in great condition and works like it came out of the box. All of the ports are tested and work perfectly. The lights are customizable via the Razer Synapse app on your PC.', 'Adorable top with a hint of lace and a key hole in the back! The pale pink is a 1X, and I also have a 3X available in white!', 'New with tags. Leather horses. Retail for  each. Stand about a foot high. They are being sold as a pair. Any questions please ask. Free shipping. Just got out of storage', 'Complete with certificate of authenticity', 'Banana republic bottoms, Candies skirt with matching blazer,Amy Byers suit, Loft bottoms and cami top.', 'Size small but straps slightly shortened to fit xs, besides that, perfect condition', 'You get three pairs of Sophie cheer shorts size small and medium girls and two sports bra/boy shorts spandex matching sets in small and medium girls. All items total retail for  in store and you can take him today for less than the price of one item at the store!)', 

In [14]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data2))

print(data_words[1:10])

[['this', 'keyboard', 'is', 'in', 'great', 'condition', 'and', 'works', 'like', 'it', 'came', 'out', 'of', 'the', 'box', 'all', 'of', 'the', 'ports', 'are', 'tested', 'and', 'work', 'perfectly', 'the', 'lights', 'are', 'customizable', 'via', 'the', 'razer', 'synapse', 'app', 'on', 'your', 'pc'], ['adorable', 'top', 'with', 'hint', 'of', 'lace', 'and', 'key', 'hole', 'in', 'the', 'back', 'the', 'pale', 'pink', 'is', 'and', 'also', 'have', 'available', 'in', 'white'], ['new', 'with', 'tags', 'leather', 'horses', 'retail', 'for', 'each', 'stand', 'about', 'foot', 'high', 'they', 'are', 'being', 'sold', 'as', 'pair', 'any', 'questions', 'please', 'ask', 'free', 'shipping', 'just', 'got', 'out', 'of', 'storage'], ['complete', 'with', 'certificate', 'of', 'authenticity'], ['banana', 'republic', 'bottoms', 'candies', 'skirt', 'with', 'matching', 'blazer', 'amy', 'byers', 'suit', 'loft', 'bottoms', 'and', 'cami', 'top'], ['size', 'small', 'but', 'straps', 'slightly', 'shortened', 'to', 'fit', 

In [15]:
len(data_words)

1482535

In [16]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [17]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words_nostops) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_nostops])  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


C:\Users\lara\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [18]:
# See trigram example
print(trigram_mod[bigram_mod[data_words_nostops[1685]]])

['lightly', 'worn', 'signs_wear', 'black', 'white', 'knee_length', 'sweater', 'could_fit', 'small', 'medium', 'possibly', 'even', 'large', 'purchased', 'karmaloop', 'clothes_accessories', 'similar', 'nordstrom', 'american_apparel', 'true_religion', 'free_people_urban', 'outfitters_zara', 'asos_topshop', 'lulus_tobi', 'american_apparel_brandy_melville', 'unif_nasty', 'gal_lf', 'forever', 'rebecca_minkoff', 'sephora', 'kate_spade', 'michael_kors', 'vince_camuto', 'victoria_secret_pink', 'vs', 'etc']


In [21]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [23]:
# Form Bigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy  download en
import en_core_web_sm
nlp = en_core_web_sm.load()

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [24]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [25]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                            alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]    

[(0,
  '0.014*"back" + 0.013*"case" + 0.010*"front" + 0.010*"look" + 0.008*"design" '
  '+ 0.008*"piece" + 0.008*"super_cute" + 0.007*"love" + 0.007*"pocket" + '
  '0.006*"long"'),
 (1,
  '0.024*"item" + 0.020*"bundle" + 0.017*"ship" + 0.015*"shipping" + '
  '0.015*"free" + 0.013*"include" + 0.013*"set" + 0.013*"make" + 0.012*"get" + '
  '0.010*"purchase"'),
 (2,
  '0.124*"new" + 0.067*"brand" + 0.026*"use" + 0.022*"come" + 0.022*"box" + '
  '0.019*"color" + 0.015*"price_firm" + 0.014*"authentic" + 0.012*"free_shipp" '
  '+ 0.012*"good"'),
 (3,
  '0.087*"size" + 0.028*"condition" + 0.027*"black" + 0.025*"great" + '
  '0.020*"small" + 0.016*"fit" + 0.015*"medium" + 0.014*"pink" + 0.014*"white" '
  '+ 0.014*"wear"')]


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.607253928342942

Coherence Score:  0.5125941023795811


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word) 
vis

In [ ]:
import pickle

with open('data_lemmatized', 'wb') as f:
    pickle.dump(data_lemmatized, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
print("dostaaaa")

In [ ]:
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("model")
lda_model.save(temp_file)

In [ ]:
with open('doc_lda.pickle', 'wb') as f:
    pickle.dump(doc_lda , f, pickle.HIGHEST_PROTOCOL)